In [1]:
import pandas as pd
from sklearn.metrics.pairwise import nan_euclidean_distances
import numpy as np

In [2]:
ratings_df=pd.read_csv('ml-100k/u.data',sep='\t',header=None,names=['user_id','item_id','rating','timestamp'])
item_df=pd.read_csv('ml-100k/u.item',sep='|',header=None,names=['movie_id' , 'movie_title' , 'release_date' , 'video_release_date' ,
              'IMDb_URL' , 'unknown' , 'Action' , 'Adventure' , 'Animation' ,
              'Childrens' , 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
              'Film-Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
              'Thriller' , 'War' , 'Western'],encoding='latin-1')
occupation_df=pd.read_csv('ml-100k/u.occupation',header=None,names=['occupation'])
user_df=pd.read_csv('ml-100k/u.user',sep='|',header=None,names=['user_id' , 'age' , 'gender' , 'occupation' , 'zip_code'])
genre_df=pd.read_csv('ml-100k/u.genre',sep='|',header=None,names=['genre' , 'genre_id']) 

In [3]:
#Find the mean age of users in each occupation
def task1():
    return user_df.groupby('occupation')[['age']].mean('age')

In [4]:
task1()

,age
occupation,
administrator,38.746835
artist,31.392857
doctor,43.571429
educator,42.010526
engineer,36.388060
entertainment,29.222222
executive,38.718750
healthcare,41.562500
homemaker,32.571429


In [5]:
#Find the names of top 20 highest rated movies. (at least 35 times rated by Users)
def task2():
    return ratings_df.merge(item_df,left_on='item_id',right_on='movie_id')[['movie_title','user_id','rating']]\
    .groupby('movie_title')\
    .agg({'user_id':'count', 'rating':'mean'})\
    .rename(columns={'user_id':'rating_count','rating':'mean_rating'})\
    .query('rating_count>=35')\
    .sort_values('mean_rating',ascending=False).head(20)

In [6]:
task2()

,rating_count,mean_rating
movie_title,,
"Close Shave, A (1995)",112,4.491071
Schindler's List (1993),298,4.466443
"Wrong Trousers, The (1993)",118,4.466102
Casablanca (1942),243,4.456790
Wallace & Gromit: The Best of Aardman Animation (1996),67,4.447761
"Shawshank Redemption, The (1994)",283,4.445230
Rear Window (1954),209,4.387560
"Usual Suspects, The (1995)",267,4.385768
Star Wars (1977),583,4.358491


In [7]:
#Find the top genres rated by users of each occupation in every age-groups. age-groups can be defined as 20-25, 25-35, 35-45, 45 and older
def task3():
    bins= [20,25,30,35,45,100]
    labels = ['20-25','25-30','30-35','35-45','45 and older']
    user=user_df.copy()
    user['age_group']=pd.cut(user['age'],bins=bins, labels=labels, right=False)
    joined_df=user.merge(ratings_df,on='user_id').merge(item_df,left_on='item_id',right_on='movie_id')[['user_id','occupation','age_group','rating','movie_title','unknown' , 'Action' , 'Adventure' , 'Animation' ,
              'Childrens' , 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
              'Film-Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
              'Thriller' , 'War' , 'Western']]
    grouped_df= joined_df.groupby(['occupation','age_group'])[['unknown' , 'Action' , 'Adventure' , 'Animation' ,
              'Childrens' , 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
              'Film-Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
              'Thriller' , 'War' , 'Western']].sum()
    grouped_df['top_rated_genre']=grouped_df.idxmax(axis=1)
    return grouped_df[['top_rated_genre']]

In [8]:
task3()

/var/folders/w8/c190fx5j08j7mz7zyp3r2l6w0000gq/T/ipykernel_75484/1043119792.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df= joined_df.groupby(['occupation','age_group'])[['unknown' , 'Action' , 'Adventure' , 'Animation' ,


top_rated_genre
occupation    age_group                   
administrator 20-25                  Drama
              25-30                  Drama
              30-35                  Drama
              35-45                  Drama
              45 and older           Drama
...                                    ...
writer        20-25                  Drama
              25-30                  Drama
              30-35                  Drama
              35-45                  Drama
              45 and older           Drama

[105 rows x 1 columns]

In [22]:
ratings_df['rating_norm']=(ratings_df['rating']-1)/4 #min-max normalization   
    #co occurance
ratings_df['occurence']=1
user_movie_matrix=ratings_df.pivot(index='user_id',columns=['item_id'],values='occurence').fillna(0)
co_occurence_matrix=np.dot(user_movie_matrix.T,user_movie_matrix)
co_occurence_df=pd.DataFrame(co_occurence_matrix,index=user_movie_matrix.columns,columns=user_movie_matrix.columns)
    

In [29]:
#given a movie, find top 10 similar movie based on user ratings
def prepare_similarity_cooccurence_data():
    ratings_df['rating_norm']=(ratings_df['rating']-1)/4 #min-max normalization
    
    #co occurance
    ratings_df['occurence']=1
    user_movie_matrix=ratings_df.pivot(index='user_id',columns=['item_id'],values='occurence').fillna(0)
    co_occurence_matrix=np.dot(user_movie_matrix.T,user_movie_matrix)
    co_occurence_df=pd.DataFrame(co_occurence_matrix,index=user_movie_matrix.columns,columns=user_movie_matrix.columns)
    
    #co-occurence threshold=50
    valid_rating_transform=co_occurence_df.applymap(lambda x:0 if x<50 else 1)

    #using nan_euclidean_distance 
    #dist(x,y) = sqrt(weight * sq. distance from present coordinates) where, weight = Total # of coordinates / # of present coordinates
    pivot_df=ratings_df.pivot(index='item_id',columns=['user_id'],values=['rating_norm'])
    nan_euclid_dist=pd.DataFrame(nan_euclidean_distances(pivot_df),columns=[i for i in range(1,len(pivot_df)+1)],index=[i for i in range(1,len(pivot_df)+1)])
    
    #similarity(x,y)=1/(1+dist(x,y)
    nan_euclid_similarity=nan_euclid_dist.applymap(lambda x:1/(1+x))
    
    #valid ones are those with co occurence>=50
    valid_nan_euclid_similarity=nan_euclid_similarity*valid_rating_transform
    return valid_nan_euclid_similarity,co_occurence_df

def find_simialar_movies(title):
    valid_nan_euclid_similarity,co_occurence_df=prepare_similarity_cooccurence_data()
    movie_id=int(item_df[item_df['movie_title']==title].movie_id.iloc[0])
    similar_movies=valid_nan_euclid_similarity[movie_id].sort_values(ascending=False)
    similar_movies=pd.DataFrame(similar_movies[similar_movies.index!=movie_id]).reset_index()[:10]
    cooccurences=pd.DataFrame(co_occurence_df[movie_id]).reset_index().rename(columns={movie_id:'strength'})
    similar_movies=similar_movies.merge(item_df, left_on='index',right_on='movie_id')[['movie_id','movie_title',movie_id]]\
        .rename(columns={movie_id:'score'})\
        .merge(cooccurences,left_on='movie_id',right_on='item_id')[['movie_title','score','strength']]
    return similar_movies

def task4():
    return find_simialar_movies('Usual Suspects, The (1995)')

In [30]:
task4()

/var/folders/w8/c190fx5j08j7mz7zyp3r2l6w0000gq/T/ipykernel_75484/3854644375.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  valid_rating_transform=co_occurence_df.applymap(lambda x:0 if x<50 else 1)
/var/folders/w8/c190fx5j08j7mz7zyp3r2l6w0000gq/T/ipykernel_75484/3854644375.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  nan_euclid_similarity=nan_euclid_dist.applymap(lambda x:1/(1+x))


,movie_title,score,strength
0,"Close Shave, A (1995)",0.129413,56.0
1,L.A. Confidential (1997),0.129262,113.0
2,Rear Window (1954),0.128344,115.0
3,Sling Blade (1996),0.128015,94.0
4,"Shawshank Redemption, The (1994)",0.123957,177.0
5,"Manchurian Candidate, The (1962)",0.121925,75.0
6,"Wrong Trousers, The (1993)",0.120901,68.0
7,Good Will Hunting (1997),0.120277,65.0
8,To Kill a Mockingbird (1962),0.117096,113.0
9,North by Northwest (1959),0.116250,103.0
